In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [3]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [4]:
#import models with pickle
import pickle
clf1 = pickle.load(open('models/catboost_model.sav', 'rb'))
clf2 = pickle.load(open('models/extra_model.sav', 'rb'))
clf3 = pickle.load(open('models/svm_model.sav', 'rb'))
clf4 = pickle.load(open('models/xgboost_model.sav', 'rb'))


In [5]:
#voting classifier
from sklearn.ensemble import VotingClassifier
eclf1 = VotingClassifier(estimators=[('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4)], voting='hard')


In [6]:
#commento perché da errore con catboost :(
'''
scores = cross_val_score(eclf1, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)
print(scores.mean())
print(scores.std())
'''

#0.6773494776392961 / 0.00782106085974223 no catboost

"\nscores = cross_val_score(eclf1, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)\nprint(scores.mean())\nprint(scores.std())\n"

In [7]:
eclf2 = VotingClassifier(estimators=[ ('catboost',clf1), ('extra', clf2), ('svm', clf3), ('xgboost', clf4)], voting='soft')
scores = cross_val_score(eclf2, x_train, np.ravel(y_train), cv=5, scoring='f1_micro' , n_jobs=8, verbose=2)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


Learning rate set to 0.235166
Learning rate set to 0.235166
Learning rate set to 0.235166
Learning rate set to 0.235163
Learning rate set to 0.235163
0:	learn: 0.6118710	total: 1.32s	remaining: 6m 35s
0:	learn: 0.6047875	total: 1.89s	remaining: 9m 24s
1:	learn: 0.6133366	total: 2.57s	remaining: 6m 23s
1:	learn: 0.6113825	total: 3.2s	remaining: 7m 56s
0:	learn: 0.6127535	total: 2.92s	remaining: 14m 33s
0:	learn: 0.6104055	total: 3.55s	remaining: 17m 40s
2:	learn: 0.6262824	total: 4.08s	remaining: 6m 43s
0:	learn: 0.6068898	total: 3.73s	remaining: 18m 34s
2:	learn: 0.6211529	total: 4.83s	remaining: 7m 58s
1:	learn: 0.6184660	total: 5.14s	remaining: 12m 45s
1:	learn: 0.6181285	total: 4.91s	remaining: 12m 11s
3:	learn: 0.6360528	total: 5.83s	remaining: 7m 11s
1:	learn: 0.6217933	total: 5.26s	remaining: 13m 3s
3:	learn: 0.6331216	total: 6.7s	remaining: 8m 15s
2:	learn: 0.6265266	total: 6.98s	remaining: 11m 31s
2:	learn: 0.6388957	total: 6.64s	remaining: 10m 57s
4:	learn: 0.6348315	total: 7.

[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed: 15.3min remaining: 23.0min


[CV] END .................................................... total time=15.3min
[CV] END .................................................... total time=15.3min
[CV] END .................................................... total time=15.3min


[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed: 15.4min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed: 15.4min finished


In [8]:
print(scores.mean())
print(scores.std())
#0.6990398567326491 / 0.01341123010967938 MinMaxScaler
#0.7064684139784948 / 0.010452913689302797 StandardScaler
#0.7154587075391008 / 0.009446903603822974 new preprocessing
#0.7318735489980451 / 0.005910129477518201 new new preprocessing

0.7318735489980451
0.005910129477518201


In [9]:
eclf2.fit(x_train, np.ravel(y_train))
y_test = eclf2.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_ensemble_soft.csv', index=False)

Learning rate set to 0.237857
0:	learn: 0.6226304	total: 533ms	remaining: 2m 39s
1:	learn: 0.6220442	total: 976ms	remaining: 2m 25s
2:	learn: 0.6325972	total: 1.43s	remaining: 2m 21s
3:	learn: 0.6384600	total: 1.91s	remaining: 2m 21s
4:	learn: 0.6443228	total: 2.4s	remaining: 2m 21s
5:	learn: 0.6533125	total: 2.85s	remaining: 2m 19s
6:	learn: 0.6585890	total: 3.31s	remaining: 2m 18s
7:	learn: 0.6667970	total: 3.76s	remaining: 2m 17s
8:	learn: 0.6714872	total: 4.2s	remaining: 2m 15s
9:	learn: 0.6732460	total: 4.62s	remaining: 2m 14s
10:	learn: 0.6794997	total: 5.08s	remaining: 2m 13s
11:	learn: 0.6879031	total: 5.58s	remaining: 2m 13s
12:	learn: 0.6898573	total: 6.08s	remaining: 2m 14s
13:	learn: 0.6931796	total: 6.56s	remaining: 2m 13s
14:	learn: 0.6992378	total: 7.01s	remaining: 2m 13s
15:	learn: 0.7027555	total: 7.48s	remaining: 2m 12s
16:	learn: 0.7084229	total: 7.95s	remaining: 2m 12s
17:	learn: 0.7138949	total: 8.37s	remaining: 2m 11s
18:	learn: 0.7164354	total: 8.77s	remaining: 2